### Imports

In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
import pathlib
import tensorflow as tf
import os
import tensorflow.keras.applications     as     applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../utilities/')
from utilities import Timer, show_image, DreamImage, DeepDream, class_names, ProgressBar, plot_confusion_matrix

### Setup

In [4]:
sns.set()

AUTOTUNE = tf.data.experimental.AUTOTUNE

width = height = 224
batch_size = 32

### Load the Model

In [6]:
model_name = 'vgg19-INet-down2-b'
base_model = load_model(f'../classification/logs/models/{model_name}.hdf5')

# Remove dropout
model = Sequential()
for layer in base_model.layers[:-1]:
    if isinstance(layer, Dropout):
        pass
    else:
        model.add(layer)

### Load the Data in Special Form

In [7]:
# Load the list of file names in the form of data_dir/class_name/file_name.jpg
data_subdir = 'test'
data_dir = pathlib.Path('../dataset/images/') / pathlib.Path(data_subdir)
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

dataset_size = len(list(list_ds))

In [8]:
# Function to get the label
#  Modification: get just the class name, not the vector
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

# Function to get the image
def decode_img(file_path):
    # Load the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)

    # Resize
    return tf.image.resize(img, [width, height])

# Preprocess
def preprocess_img(img, label, file_path):
    return applications.vgg19.preprocess_input(img), label, file_path

In [9]:
# Combine both processes
def process_path(file_path):
    return decode_img(file_path), get_label(file_path), file_path

In [10]:
def embed_image(img_batch, label_batch, file_path_batch):
    return model(img_batch), label_batch, file_path_batch

In [11]:
# Create the dataset
ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
ds = ds.map(preprocess_img)
ds = ds.batch(batch_size).prefetch(buffer_size = AUTOTUNE)
ds = ds.map(embed_image)
ds = ds.cache()

### Compute Embedding on the Dataset

In [12]:
bar = ProgressBar(dataset_size // batch_size + 1)

In [13]:
class_batches     = []
label_batches     = []
file_path_batches = []
bar.start()
iterations = 0
for class_batch, label_batch, file_path_batch in ds:
    class_batches.append(class_batch)
    label_batches.append(label_batch)
    file_path_batches.append(file_path_batch)
    iterations+=1
    bar.update(iterations)

[==================================================>] 92/92 7.0s per loop          


### Extract Embeddings into a Convenient Numpy Array

In [16]:
# Convert to a single array
def concat_batches(batch_list):
    batch_list = [tensor.numpy() for tensor in batch_list]
    return np.concatenate(batch_list)

embedding  = concat_batches(class_batches)
labels     = concat_batches(label_batches)
file_paths = concat_batches(file_path_batches)

df = pd.concat([pd.Series(labels, name = 'artist'), pd.Series(file_paths, name = 'file_path'), pd.DataFrame(embedding)], axis = 'columns')
df.to_csv(f'{model_name}-embedding.csv')